<a href="https://colab.research.google.com/github/Aatish1066/python-code/blob/main/JUIT_DSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment **1**_201536


In [ ]:
!pip install simpy


In [ ]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We yield the process that process() returns
             # to wait for it to finish
             yield self.env.process(self.charge(charge_duration))

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
        yield self.env.timeout(duration)
env = simpy.Environment()
car = Car(env)
env.run(until=15)
        

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


# Experiment **2**_201536

In [ ]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

class Car(object):
     def __init__(self, env):
         self.env = env
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We may get interrupted while charging the battery
             try:
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:
                 # When we received an interrupt, we stop charging and
                 # switch to the "driving" state
                 print('Was interrupted. Hope, the battery is full enough ...')

             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)
     def charge(self, duration):
        yield self.env.timeout(duration)   
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)         

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


#Experiment_3_201536
#Task
Electric cars drive to the Battery Charging Station (BCS), wait for their slot, charge and leave. When it starts driving to the BCS, it will print the current simulation time. The car charges at a battery charging station if charging slots are avilable. Simulate four cars with charge duration 5 and driving duration 2, 4, 6, 8 respectively using **SimPy**


In [ ]:
 def car(env, name, bcs, driving_time, charge_duration):
     # Simulate driving to the BCS
     yield env.timeout(driving_time)

     # Request one of its charging spots
     print('%s arriving at %d' % (name, env.now))
     with bcs.request() as req:
         yield req

         # Charge the battery
         print('%s starting to charge at %s' % (name, env.now))
         yield env.timeout(charge_duration)
         print('%s leaving the bcs at %s' % (name, env.now))

In [ ]:
import simpy
env = simpy.Environment()
def car(env):
    while True:
         print('Start parking at %d' % env.now)
         parking_duration = 5
         yield env.timeout(parking_duration)

         print('Start driving at %d' % env.now)
         trip_duration = 2
         yield env.timeout(trip_duration)

env.process(car(env))

env.run(until=15)


Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


#Experiment 4_201536

In [ ]:
import simpy
env = simpy.Environment()
def trafflight(env):
     while True:
          print('signal is green at %d'%env.now)
          green_duration = 25
          yield env.timeout(green_duration)
          print('signal is yellow at %d'%env.now)
          yellow_duration = 15
          yield env.timeout(yellow_duration)
          print('signal is red at %d'%env.now)
          red_duration = 60
          yield env.timeout(red_duration)

env.process(trafflight(env))
env.run(until=200)

signal is green at 0
signal is yellow at 25
signal is red at 40
signal is green at 100
signal is yellow at 125
signal is red at 140


##experiment_6_201536
Using SimPy model a bank counter and customers arriving at random times. Each customer has
a certain patience. Customers waits to get to the counter until they are at the end of their tether.
If they gets to the counter, they uses it for a while before releasing it.


In [ ]:
import random                                                                   #importing required libraries
import simpy
NEW_CUSTOMERS =5                                                                #no of customers that are going to enter the bank in the stimulation
INTERVAL_CUSTOMERS =10.0  
MIN_PATIENCE=1
MAX_PATIENCE=3

def customer(env,name,counter,time_in_bank):
  arrive=env.now
  print('%7.4f: %s Arrived' % (arrive, name))
  with counter.request() as req:
      patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
      # Wait for the counter or abort at the end of our tether
      results = yield req | env.timeout(patience)
      
      
      
      wait = env.now - arrive
      if req in results:
          # We got to the counter
          print('%7.4f: %s Waited %6.3f' % (env.now, name, wait))

          tib = random.expovariate(1.0 / time_in_bank)

          yield env.timeout(tib)
          print('%7.4f: %s Finished' % (env.now, name))

      else:
        print('%7.4f: %s RENEGED after %6.3f' % (env.now, name, wait))
def source(env, number, interval, counter):
    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, 'Customer%02d' % i, counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

 #Setup and start the simulation
print('Bank renege')
env = simpy.Environment()
# Start processes and run
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()        


Bank renege
 0.0000: Customer00 Arrived
 0.0000: Customer00 Waited  0.000
 8.0727: Customer01 Arrived
 9.2537: Customer01 RENEGED after  1.181
 9.7131: Customer00 Finished
11.2319: Customer02 Arrived
11.2319: Customer02 Waited  0.000
18.8095: Customer02 Finished
23.1522: Customer03 Arrived
23.1522: Customer03 Waited  0.000
26.8330: Customer03 Finished
32.1350: Customer04 Arrived
32.1350: Customer04 Waited  0.000
37.7784: Customer04 Finished


# experiment the lab

In [1]:
import simpy
import random
def lab(env,att):
  no_students=att
  labhr=2
  max_att=29
  min_att=5
  chance=random.uniform(0,1)
  dec=env.now
  if(no_students>15 and chance>0.5 ):
    print('attend lab no chutti')
  else:
    print('because are present no lab today hence chutti') 
  
env = simpy.Environment()
env.process(lab(env,12))
env.run()

    


ModuleNotFoundError: ignored

## Task
Simulate a workshop with n identical machines. A stream of jobs arrives at the workshop. Each machine breaks down periodically. Repairs are carried out by one repairman. The repairman has other, less important tasks to perform, too. Broken machines preempt theses tasks. The repairman continues them when he is done with the machine repair. The workshop works continuously.  

A machine has two processes: working implements the actual behaviour of the machine (producing parts). break_machine periodically interrupts the working process to simulate the machine failure.  

The repairman’s other job is also a process (implemented by other_job). The repairman itself is a PreemptiveResource with a capacity of 1. The machine repairing has a priority of 1, while the other job has a priority of 2 (the smaller the number, the higher the priority)